<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Telco Network Coverage Analysis</b>
</header>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'>
Driving Spatial Insights for Network Optimization to optimize network layouts, minimize construction costs and plan least-cost network reducing CapEx
</p>
<p style = 'font-size:16px;font-family:Arial'>
Mobile communication is one of the technology that is essential in the present era. Telecommunication towers which is a supporting factor of mobile telecommunication services are constantly constructed as a form of community needs in communication.
</p>
<p style = 'font-size:16px;font-family:Arial'>
To achieve these objectives, the calculation of the number of mobile subscribers and traffic capacity into a method of planning is needed to determine the number of base stations and telecommunication towers.

</p>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Analyzing New Potential Coverage Area using GIS</b></p>

<p style = 'font-size:16px;font-family:Arial'>With geographical information available in the form of Geographic Information Systems (GIS) can provide information about the reach and area of signal strength for each telecommunication tower. Information about prospective customers who are in an area that has not been reached by the existing tower so that it can be a reference for the addition of towers, this information system can facilitate the service provider to determine the service users who will subscribe.
</p>

<hr>
<p style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>1. Connect to Vantage</b></p>


<p style = 'font-size:16px;font-family:Arial'>Let's start by importing the libraries needed.</p>

In [ ]:
#import libraries
import geopandas as gpd
import matplotlib.pyplot as plt 
import pandas as pd
import getpass
import warnings


warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

from teradataml import *
import plotly.express as px
import plotly.graph_objects as go
import json
from pandas import json_normalize
from  ipywidgets import widgets, interact

display.max_rows = 5 

<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then <b>use down arrow</b> to go to next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql("SET query_band='DEMO=Telco_Network_Coverage_PY_SQL.ipynb;' UPDATE FOR SESSION;")

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You have the option of either running the demo using foreign tables to access the data without using any storage on your environment or downloading the data to local storage which may yield somewhat faster execution, but there could be considerations of available storage. There are two statements in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_TelcoNetwork_cloud');"
 # takes about 30 seconds, estimated space: 0 MB
%run -i ../run_procedure.py "call get_data('DEMO_TelcoNetwork_local');" 
# takes about 30 seconds, estimated space: 3 MB

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<p style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>2. Initial Data Sets</b></p>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Cell Tower</b></p>
<p style = 'font-size:16px;font-family:Arial'>Geographical location (Longitude and Latitude) of Cell Towers with additional details of the cell towers were given in this file. Let's see how the data looks like.</p>

In [ ]:
res = DataFrame(in_schema("DEMO_TelcoNetwork", "Cell_Towers"))
res

<p style = 'font-size:16px;font-family:Arial'>We can see that the data has details of the cell towers. Let's visualize this data to better understand the location of towers.</p>

In [ ]:
#cell tower sites
res.drop(['radio_type','radio_freq','degree_facing','cell_id','postal_area_code','postal_area_name','dist_kms_from_map_centroid','cell_geom'], axis=1)
cell_tower = res[(res.cell_lon < 152)].to_pandas()

In [ ]:
fig = px.scatter_mapbox(cell_tower, lat="cell_lat", lon="cell_lon", hover_name="cell_site_name", 
                        hover_data=["site_num"],
                        color_discrete_sequence=["black"], zoom=7, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

<p style = 'font-size:16px;font-family:Arial'>Here we can see the tower locations on the map.</p> 

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Aggregated Customer Experience</b></p>
<p style = 'font-size:16px;font-family:Arial'>Geographical points (Longitude and Latitude) of customer experience using RSCP/signal dBm values were given in this file. Let's see how the data looks like.</p>

In [ ]:
res = DataFrame(in_schema("DEMO_TelcoNetwork", "Cust_Experience"))
res

<p style = 'font-size:16px;font-family:Arial'>
BTS antenna Coverage Area is a distribution of signals from the antenna on the earth surface. The aspects affecting the coverage area are the type of antenna, the type of environment, the propagation model, the cell radius and the power budget.</p>
<p style = 'font-size:16px;font-family:Arial'>
The Signal strength is measured in RSCP(Received Signal Code Power) which denoted the power measured by a receiver on a particular physical communication channel. It is used as an indication of signal strength, as a handover criterion.</p>
<p style = 'font-size:16px;font-family:Arial'>Let's plot the information in the map for better understanding.</p>

In [ ]:
#dbm point values
query = '''
select legend,rscpavg,Cast(POINT.ST_X() as Decimal(12,8)) lon,
Cast(POINT.ST_Y() as Decimal(12,8)) lat, 
case  when (RSCPAVG <= -101) Then 'poor' 
    when  (RSCPAVG between -100 and -86) Then 'good'
    when  (RSCPAVG >= -85 ) Then 'excellent'
    else 'na' 
end as "signal"
    from DEMO_TelcoNetwork.Cust_Experience;
'''
nw = DataFrame.from_query(query)
nw

In [ ]:
nw=nw.to_pandas(all_rows=True)
fig = px.scatter_mapbox(nw, lat="lat", lon="lon",  
                        hover_data=["RSCPAVG"],
                        color="signal"  ,
                        color_discrete_map={
                                             "poor": "red",
                                             "good": "yellow",
                                             "excellent": "green",
                                             "na": "black"
                                          } ,
                        labels={"signal": "RSCPAVG" },
                        zoom=11, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

<p style = 'font-size:16px;font-family:Arial'>In this visualization we have divided our signal strengths into categories of excellent (rscpavg greater than -85) /good (rscpavg between -100 and -80) /poor (rscpavg less than -100) based on the rscpavg value we have recieved from the corresponding location.</p>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Coveage Map</b></p>
<p style = 'font-size:16px;font-family:Arial'>GeoJson file of  a geographical area which covers the network area under consideration was given.</p>


In [ ]:
with open ("./data/main_map.geojson",'r') as infile:
    counties = json.load(infile)

In [ ]:
#for colorcoding different coverage area
df1 = json_normalize(counties['features'])
c_code = df1["properties.SITEID"].unique()
df2 = pd.DataFrame(c_code)
df3 = []
for i in range(len(c_code)):
    df3.append([c_code[i],i+1])
df = pd.DataFrame(df3, columns = ['siteid', 'colourcode'])


In [ ]:
fig = px.choropleth_mapbox(df, geojson=counties,
                           locations='siteid',
                           featureidkey='properties.SITEID',
                           color='colourcode',
                           hover_name='siteid',
                           range_color=(0,30),
                           mapbox_style="open-street-map",
                           color_continuous_scale="Viridis",
                           zoom=10,height=300,center ={"lat":-33.06363, "lon":151.74148},
                           opacity=0.5
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


<p style = 'font-size:16px;font-family:Arial'>In the visualization we can see the area marked on the map. We can also load this GeoJson file in Vantage using GeoImport Utility</p>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Customer Navigation</b></p>
<p style = 'font-size:16px;font-family:Arial'>This file contains the Customer navigation steps for 364 journeys. It is the superset data and contains details like incremental journey sequence, incremental timestamp for each step, values of signal_dBm, geographical co-ordinates of each journey, mobile app,throughput etc. Let's check the data of this file. </p>

In [ ]:
res = DataFrame(in_schema("DEMO_TelcoNetwork", "Cust_Journey"))
res

<p style = 'font-size:16px;font-family:Arial'>From this table we can create 1 row for each customer journey trajectory using <b>Vantage ClearScape Analytics UAF(unbounded array framework)</b>. <b>TD_TRACKINGOP</b> is a multi-dimensional function for geospatial data in UAF which can calculate trip distance, speed, time for a trip. Let's see how we can use that in our given data.<p>
    

In [ ]:
# Create a volatile table for selected customers to be used in UAF function 
qry = '''
create volatile table uaf_customer as 
(select customer_id, 
        journey_id, 
        journey_seq,
        signal_dBm,
        journey_dttm as arv_ts, 
        (arv_ts + interval '1' second) as dep_ts ,
        cast(journey_geom as SYSUDTLIB.ST_GEOMETRY(400)) as journey_geom 
 from DEMO_TelcoNetwork.Cust_Journey
 where customer_id in ('CID-0005005','CID-0005073','CID-0005233','CID-0005243')
) with data on commit preserve rows;
'''

execute_sql(qry)


<p style = 'font-size:16px;font-family:Arial'>The data in the volatile table looks like below</p>

In [ ]:
res = DataFrame("uaf_customer")
res[(res.customer_id == 'CID-0005243')]

<p style = 'font-size:16px;font-family:Arial'>Now let's use the TD_TRACKINGOP function which stores the result in <b>Analytical Result Table (ART)</b></p>

In [ ]:
qry = '''
EXECUTE FUNCTION INTO VOLATILE ART(cust_travel)
TD_TRACKINGOP( 
 SERIES_SPEC( TABLE_NAME(uaf_customer),ROW_AXIS(TIMECODE(arv_ts)),
 SERIES_ID(customer_id, journey_id), 
 PAYLOAD (FIELDS (arv_ts,dep_ts, journey_geom),
 CONTENT(MULTIVAR_ANYTYPE)) ),
 FUNC_PARAMS(DISTANCE(1), SPEED(1), TIME_SPENT(1), METRIC(1))
);
'''

execute_sql(qry)

<p style = 'font-size:16px;font-family:Arial'>Let's check the output of function</p>

In [ ]:
res = DataFrame("cust_travel")
res[(res.customer_id == 'CID-0005243')]

<p style = 'font-size:16px;font-family:Arial'>From the above output, we can see the customer trajectory along with his speed and total trip time and trip distance. We can also plot the customer trajectory for better visualization.</p>

In [ ]:
#customer navigation path
qry = '''
select customer_id,journey_id,journey_seq,signal_dBm,Cast(journey_geom.ST_X() as Decimal(12,8)) lon,
Cast(journey_geom.ST_Y() as Decimal(12,8)) lat,
case  when (signal_dBm <= -101) Then 0 /*poor*/
    when  (signal_dBm between -100 and -86) Then 1 /*good*/
    when  (signal_dBm >= -85 ) Then 2  /*excellent*/
    else 0 
end as color_scale 
from uaf_customer ;
'''

nw_path2 = DataFrame.from_query(qry)
nw_path2.sort(['customer_id','journey_id','journey_seq'])


In [ ]:
nw_path2=nw_path2.to_pandas(all_rows=True)
def plot_map(customer):    
    
    custom_colors = [
    [0, '#FF0000'], # red
    [0.5, '#FFEA00'], # yellow
    [1, '#028A0F'] # green
    ]
    
    plt.figure(figsize=(8, 6))
    df = nw_path2[nw_path2['customer_id'] == customer ]
    fig = px.line_mapbox(df, lat="lat", lon="lon",
                     hover_data=["journey_id", "customer_id"],
                     color="journey_id"  ,
                     zoom=8, height=300)
    fig.add_trace(go.Scattermapbox(
    name = "signal",
    mode = "markers",
    lat = df.lat.tolist(),
    lon = df.lon.tolist(),
    hoverinfo='text',
    hovertemplate= ['<b>Signal_dBm:</b>:' + str(df.iloc[i, 3]) for i in range(df.shape[0])],
    marker_color=df['color_scale'] ,
    marker=dict(
        color=[1, 2, 3],
        colorscale=custom_colors,
        size = 8
    )    

))

    fig.update_layout(mapbox_style="open-street-map" , #"stamen-terrain",
                  mapbox_zoom=9,
                  mapbox_center_lon=151.5,
                  mapbox_center_lat=-33.1,
                  margin={"r":0,"t":0,"l":0,"b":0}
                 )
    
       
    return fig

# Create the dropdown widget
customer = ["CID-0005005","CID-0005073", "CID-0005233","CID-0005243"]
customer_dropdown = widgets.Dropdown(options=customer, description='Customer:', value='CID-0005005')

# Call the plot_clusters function with the selected dropdown options
def update_plot(customer):    
    plot_map(customer).show()
    
widgets.interact(update_plot, customer=customer_dropdown)



<p style = 'font-size:16px;font-family:Arial'>Please use the dropdown to see the navigation paths for customer. We have also marked the signal strength at points where data was captured based on the bDm values.

<p style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>3. Cleanup</b></p>

<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_TelcoNetwork');" 
#Takes 10 seconds

In [ ]:
remove_context()

<p style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>4. Conclusion</b></p>
<p style = 'font-size:16px;font-family:Arial'>Tower map distribution in digital form can be used as one of the tools that can facilitate the operational field for monitoring, organizing BTS assets owned by the company in map view.</p>
<p style = 'font-size:16px;font-family:Arial'><b>Vantage Clearscape Analytics</b> can do complex geospatial calculations very easily and the results can be exported to visualization tools. The potential analysis of service area maps can be used as a reference by stakeholders for making crucial decisions in terms of telecommunications development and expansion of the business.</p>


<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Reference Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'> 
       <li>Teradata Vantage™ - Unbounded Array Framework Time Series Reference: <a href = 'https://docs.teradata.com/r/Teradata-VantageTM-Unbounded-Array-Framework-Time-Series-Reference/Unbounded-Array-Framework '>https://docs.teradata.com/r/Teradata-VantageTM-Unbounded-Array-Framework-Time-Series-Reference/Unbounded-Array-Framework </a></li>    
  <li>Teradata® Geospatial Utilities User Guide: <a href = 'https://docs.teradata.com/r/Teradata-Geospatial-Utilities-User-Guide/June-2022/Teradata-Geospatial-Utilities-Overview/Welcome-to-Teradata-Tools-and-Utilities-Teradata-Geospatial-Utilities-User-Guide'>https://docs.teradata.com/r/Teradata-Geospatial-Utilities-User-Guide/June-2022/Teradata-Geospatial-Utilities-Overview/Welcome-to-Teradata-Tools-and-Utilities-Teradata-Geospatial-Utilities-User-Guide</a></li>
  <li>Python Pandas Reference: <a href = 'https://pandas.pydata.org/docs/user_guide/index.html'>https://pandas.pydata.org/docs/user_guide/index.html</a></li>
<li>Plotly Reference: <a href = 'https://plotly.com/'>https://plotly.com/</a></li>
      
</ul>


<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">©2023 Teradata. All Rights Reserved</footer>